In [1]:
import torch
import pandas as pd
import numpy as np
import pytorch_transformers

folder = 'd:\git\project1\data'

In [2]:
model_class = pytorch_transformers.BertModel
tokenizer_class = pytorch_transformers.BertTokenizer
pretrained_weights = 'bert-base-uncased'

model = model_class.from_pretrained(pretrained_weights)

In [3]:
%%time
tokens_padded = pd.read_csv(f'{folder}/tweets_change_padded.csv')
tokens_padded

Wall time: 13.9 s


,retweets,date,time,change_1_direct,change_3_direct,change_7_direct,change_30_direct,change_90_direct,0,1,...,90,91,92,93,94,95,96,97,98,99
0,7.0,1558465200,45736,1.0,1.0,1.0,1.0,1.0,101,2190,...,0,0,0,0,0,0,0,0,0,0
1,68.0,1558897200,29586,1.0,1.0,1.0,1.0,1.0,101,2978,...,0,0,0,0,0,0,0,0,0,0
2,4.0,1558810800,75329,1.0,1.0,1.0,1.0,1.0,101,1002,...,0,0,0,0,0,0,0,0,0,0
3,85.0,1556737200,63389,1.0,1.0,0.0,1.0,1.0,101,16770,...,0,0,0,0,0,0,0,0,0,0
4,8.0,1558897200,5857,1.0,1.0,1.0,1.0,1.0,101,2978,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807251,15.0,1574449200,23952,0.0,0.0,0.0,0.0,0.0,101,100,...,0,0,0,0,0,0,0,0,0,0
807252,50.0,1574449200,52538,0.0,0.0,0.0,0.0,0.0,101,2057,...,0,0,0,0,0,0,0,0,0,0
807253,940.0,1574017200,36668,1.0,0.0,0.0,1.0,0.0,101,8819,...,2050,2620,102,0,0,0,0,0,0,0
807254,24.0,1574449200,55448,0.0,0.0,0.0,0.0,0.0,101,1002,...,0,0,0,0,0,0,0,0,0,0


In [4]:
%%time
padded = np.array(tokens_padded.iloc[:,8:])
attention_mask = np.where(padded != 0, 1, 0)
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

input_ids

Wall time: 2 s


tensor([[ 101, 2190, 4328,  ...,    0,    0,    0],
        [ 101, 2978, 3597,  ...,    0,    0,    0],
        [ 101, 1002, 2980,  ...,    0,    0,    0],
        ...,
        [ 101, 8819, 2003,  ...,    0,    0,    0],
        [ 101, 1002, 3263,  ...,    0,    0,    0],
        [ 101, 3407, 1001,  ...,    0,    0,    0]])

In [5]:
def save_last_hidden(start, count):
    start_index = start*count
    end_index = (start+1)*count
    last_hidden_states = model(input_ids[start_index:end_index, :], attention_mask=attention_mask[start_index:end_index, :])
    last_hidden_states_df = pd.DataFrame(last_hidden_states[0][:,0,:].numpy())
    #print(np.array(tokens_padded.loc[start_index:end_index-1, ['retweets']]))
    #last_hidden_states_df['retweet'] = np.array(tokens_padded.loc[start_index:end_index-1, ['retweets']])
    #last_hidden_states_df['time'] = np.array(tokens_padded.loc[start_index:end_index-1, ['time']])
    #last_hidden_states_df['date'] = np.array(tokens_padded.loc[start_index:end_index-1, ['date']])
    #last_hidden_states_df['change_1_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_1_direct']])
    #last_hidden_states_df['change_3_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_3_direct']])
    #last_hidden_states_df['change_7_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_7_direct']])
    #last_hidden_states_df['change_30_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_30_direct']])
    #last_hidden_states_df['change_90_direct'] = np.array(tokens_padded.loc[start_index:end_index-1, ['change_90_direct']])
    #print(last_hidden_states_df)
    last_hidden_states_df.to_csv(f'{folder}/last_hs_tweets_ch_{start_index}.csv', index=False)

In [ ]:
%%time
rows_count = input_ids.shape[0]
chank_size = 1000
start = 413
for chank_num in range (start, (int)(rows_count/chank_size)+1):
    print(chank_num)
    with torch.no_grad():
        save_last_hidden(chank_num, chank_size)
        

413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438


In [12]:
%%time
with torch.no_grad():
    #save_last_hidden(251, 1000)

Wall time: 12.1 s
